In [58]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta
from vnstock3 import Vnstock

# Setting

In [59]:
rf = {'2019': 0.0451, '2020': 0.0286, '2021':0.023,'2022': 0.0335,'2023':0.0321}
rf = pd.Series(rf)
rf_cal = rf.mean()
rf_2022 = 0.0335
rf_cal

0.03246

In [60]:
pd.set_option('display.max_columns', None) 

In [61]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_SLOW_PERIOD = 26
MACD_FAST_PERIOD = 12
MACD_SIGNAL_PERIOD = 9
initial_investment = 100_000_000
# backup_amount_initial = 40_000_000

In [62]:
win_rate = 0.5711907018
loss_rate = 1 - win_rate
mean_profit = 0.3493357553
mean_loss = 0.1397008027
stop_loss = 0.08

In [63]:
def kelly_criterion(p, q, profit, loss):
    b = (profit * 160_000_000)/(loss * 160_000_000)
    f = (b*p - q)/b
    return f

In [64]:
f =kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)
f

0.39970810302530857

In [65]:
high_ESG_group = ['CTD', 'DHG', 'DPM', 'FPT', 'GAS', 'MBB']
low_ESG_group = ['NVL', 'PNJ', 'REE', 'SBT', 'SSI', 'STB', 'VIC', 'VNM']
non_ESG_group = ['CII', 'CTG', 'EIB', 'GMD', 'HDB', 'HPG', 'MSN', 'MWG', 'ROS', 'SAB', 'TCB', 'VCB', 'VHM', 'VJC', 'VPB', 'VRE']


In [66]:
companies = ['VCB']

# Calculate indicators

In [67]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)

    return df

In [68]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

In [69]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [70]:
def calculate_daily_risk_free_rate(annual_rate_decimal, trading_days=252):
    # Convert percentage to decimal
    # Calculate daily risk-free rate using the compound formula
    daily_rate = (1 + annual_rate_decimal) ** (1 / trading_days) - 1
    
    # Alternatively, for a simple calculation:
    # daily_rate = annual_rate_decimal / trading_days
    
    return daily_rate

# Example usage
annual_yield = 0.0326  # Example annual yield of 3%
daily_risk_free_rate = calculate_daily_risk_free_rate(rf_cal)
daily_risk_free_rate_2022 = calculate_daily_risk_free_rate(rf_2022)


# Backtest

In [71]:
def simulate_investment(
    ticker, win_rate, loss_rate, mean_profit, mean_loss, 
    sell_fraction, start_date, end_date, f_star=1
):
    try:
        # Initialize trade counters and portfolio metrics
        number_of_buying_trades = 0
        number_of_selling_trades = 0
        cash = initial_investment
        holdings = 0
        portfolio_values = []

        # Load stock data and calculate indicators
        data = vn.stock_historical_data(ticker, start_date, end_date, resolution='1D', type='stock', source='TCBS')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)
        data = data.dropna(subset=['time'])
        data = data.drop_duplicates(subset=['time', 'open', 'high', 'low', 'close', 'volume'], keep='first')
        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i, current_date in enumerate(data.index):
            current_price = data['close'].iloc[i]

            # Handle pending T+2 settlements
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                portfolio_values.append(cash + holdings * current_price)
                continue

            # Buy if there's a buy signal and cash allows
            if current_date in buy_signals:
                allocation = cash * f_star
                shares_to_buy = int(allocation // current_price)
                total_cost = shares_to_buy * current_price
                if shares_to_buy > 0 and cash >= total_cost:
                    cash -= total_cost
                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    pending_buy_shares[settlement_date] = pending_buy_shares.get(settlement_date, 0) + shares_to_buy
                    last_buy_price = current_price
                    number_of_buying_trades += 1

            # Sell if there's a sell signal and holdings allow
            if holdings > 0 and current_date in sell_signals:
                shares_to_sell = int(holdings * sell_fraction)
                revenue = shares_to_sell * current_price
                holdings -= shares_to_sell
                settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                pending_sell_revenue[settlement_date] = pending_sell_revenue.get(settlement_date, 0) + revenue
                number_of_selling_trades += 1

            # Update portfolio value
            portfolio_values.append(cash + holdings * current_price)

        # Finalize portfolio values including pending settlements
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)
            portfolio_values.append(cash + holdings * data['close'].iloc[-1])
            final_date += BDay(1)

        # Adjust portfolio values to match data index length
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))
    
        # Add portfolio values to data frame
        data['Portfolio_Value'] = portfolio_values
        data['value'] = portfolio_values
        data['Number_of_Buying_Trades'] = number_of_buying_trades
        data['Number_of_Selling_Trades'] = number_of_selling_trades

        # Set `value` to NaN if `Signal` is non-zero, for both current and next row
        for i in range(len(data) - 1):  # Avoid index out of bounds for i+1
            if data['Signal'].iloc[i] != 0:  # Use iloc to access a single value
                data.loc[data.index[i], 'value'] = np.nan
                data.loc[data.index[i + 1], 'value'] = np.nan
                

        # Forward fill `value` where needed
        data['value'] = data['value'].ffill()

        # Calculate returns and performance metrics
        data['Daily_Return'] = data['value'].pct_change()
        data['Accumulated_Profit'] = data['value'] - initial_investment
        data['Running_Max'] = data['value'].cummax()  # Track the running max portfolio value
        data['Drawdown'] = (data['value'] - data['Running_Max']) / data['Running_Max']  # Calculate drawdown
        
        return data

    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()


In [72]:
def calculate_sharpe_ratio(data, risk_free_rate=0.01):
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Daily_Return'].dropna()

    # Calculate average return and standard deviation of returns
    average_return = daily_returns.mean()
    std_deviation = daily_returns.std()

    # Calculate the Sharpe Ratio
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation if std_deviation > 0 else np.nan

    return sharpe_ratio

In [73]:
def calculate_sortino_ratio(data,rf=0.01):  # Target return can be set to risk-free rate
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Daily_Return'].dropna()

    # Calculate average return
    average_return = daily_returns.mean()

    # Calculate downside returns (returns below the target return)
    downside_returns = daily_returns[daily_returns < rf]

    # Calculate downside deviation
    downside_deviation = downside_returns.std() if not downside_returns.empty else np.nan

    # Calculate the Sortino Ratio
    sortino_ratio = (average_return - rf) / downside_deviation if downside_deviation > 0 else np.nan

    return sortino_ratio

thêm phần điều chỉnh kelly trong code phía dưới

In [74]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, \
                                sell_fraction, start_date, end_date, rf):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, \
                                     sell_fraction, start_date=start_date, end_date=end_date, f_star=f)
        if not result.empty:
            # Calculate the Sharpe Ratio for the result
            sharpe_ratio = calculate_sharpe_ratio(result,risk_free_rate=rf)
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / initial_investment * 100,
                'Number of Buying Trades': result['Number_of_Buying_Trades'].max(),
                'Number of Selling Trades': result['Number_of_Selling_Trades'].max(),
                'Sharpe Ratio': sharpe_ratio,
                'Sortino Ratio': calculate_sortino_ratio(result,rf=rf),
                'MDD': result['Drawdown'].min(),
            })
    return pd.DataFrame(results)

# Kelly 2019-2024

In [75]:
sell_fraction = 1

In [76]:
 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss,\
                                        sell_fraction, start_date='2018-12-28' ,end_date='2024-01-05', rf=daily_risk_free_rate)
high_esg = pd.DataFrame(results_df)
print(high_esg)
average_rate_of_return = results_df['Rate of Return'].mean()
average_profit = results_df[results_df['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = results_df[results_df['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}','avg sharpe:',results_df['Sharpe Ratio'].mean(),'avg sortino:',results_df['Sortino Ratio'].mean())

Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD              156938530    56938530.0       56.938530   
1     DHG              104949650     4949650.0        4.949650   
2     DPM              165181410    65181410.0       65.181410   
3     FPT              159132027    59132027.0       59.132027   
4     GAS              137655590    37655590.0       37.655590   
5     MBB              134790290    34790290.0       34.790290   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                       42                        39      0.032742   
1                       44                        40     -0.015725   
2              

In [77]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss,\
                                             sell_fraction, start_date='2018-12-29', end_date='2024-01-05', \
                                                rf=daily_risk_free_rate)
low_esg_kelly = pd.DataFrame(low_esg_kelly)
print(low_esg_kelly)
average_rate_of_return = low_esg_kelly['Rate of Return'].mean()
average_profit = low_esg_kelly[low_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly[low_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL              151349229    51349229.0       51.349229   
1     PNJ              112851000    12851000.0       12.851000   
2     REE              110446340    10446340.0       10.446340   
3     SBT              125752870    25752870.0       25.752870   
4     SSI              205972160   105972160.0      105.972160   
5     STB              155582580    55582580.0       55.582580   
6     VIC              111984550    11984550.0       11.984550   
7     VNM               82395111   -17604889.0      -17.

In [78]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2018-12-29', end_date='2024-01-05', rf=daily_risk_free_rate)
non_esg_kelly = pd.DataFrame(non_esg_kelly)
print(non_esg_kelly)
average_rate_of_return = non_esg_kelly['Rate of Return'].mean()
average_profit = non_esg_kelly[non_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly[non_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error

In [79]:
high_esg['ESG'] ='High'
low_esg_kelly['ESG'] = 'Low'
non_esg_kelly['ESG'] = 'Non'

In [80]:
merged = pd.concat([high_esg, low_esg_kelly, non_esg_kelly]) 
merged['period'] = '2019-2024' 

# KELLY 2022

In [81]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
high_esg_2022 = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
high_esg_2022 = pd.DataFrame(high_esg_2022)
print(high_esg_2022)
average_rate_of_return = high_esg_2022['Rate of Return'].mean()
average_profit = high_esg_2022[high_esg_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = high_esg_2022[high_esg_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
high_esg_2022['ESG'] ='High'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD              100810550      810550.0         0.81055   
1     DHG               93147170    -6852830.0        -6.85283   
2     DPM               95260550    -4739450.0        -4.73945   
3     FPT              108893350     8893350.0         8.89335   
4     GAS              104567690     4567690.0         4.56769   
5     MBB              101609360     1609360.0         1.60936   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        8                         8     -0.005649   
1                        8                         7     -0.127515   
2                    

In [82]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly_2022 = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
low_esg_kelly_2022 = pd.DataFrame(low_esg_kelly_2022)
print(low_esg_kelly_2022)
average_rate_of_return = low_esg_kelly_2022['Rate of Return'].mean()
average_profit = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
low_esg_kelly_2022['ESG'] = 'Low'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Error occurred for PNJ: Out of bounds nanosecond timestamp: 9223545600000000000
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL              103285500     3285500.0        3.285500   
1     REE               96529470    -3470530.0       -3.470530   
2     SBT              104998160     4998160.0        4.998160   
3     SSI               97988360    -2011640.0       -2.011640   
4     STB              106453756     6453756.0        6.453756   
5     VIC               98264100    -1735900.0       -1.735900   
6     VNM              100251120      251120.0    

In [83]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly_2022 = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
non_esg_kelly_2022 = pd.DataFrame(non_esg_kelly_2022)
print(non_esg_kelly_2022)
average_rate_of_return = non_esg_kelly_2022['Rate of Return'].mean()
average_profit = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
non_esg_kelly_2022['ESG'] = 'Non'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for ROS: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for SAB: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for TCB: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit

In [84]:
backtest_multiple_companies(['PNJ','SAB','VPB'], win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)

Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for PNJ: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for SAB: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for VPB: 'NoneType' object is not subscriptable


""


In [85]:
merged_2022 = pd.concat([high_esg_2022, low_esg_kelly_2022, non_esg_kelly_2022])
merged_2022['period'] = '2022-2023'
merged_2022

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,100810550,810550.0,0.810550,8,8,-0.005649,-0.007848,-0.097149,High,2022-2023
1,DHG,93147170,-6852830.0,-6.852830,8,7,-0.127515,-0.161295,-0.071959,High,2022-2023
2,DPM,95260550,-4739450.0,-4.739450,8,8,-0.036775,-0.049215,-0.111606,High,2022-2023
3,FPT,108893350,8893350.0,8.893350,5,5,0.044085,0.066771,-0.053444,High,2022-2023
4,GAS,104567690,4567690.0,4.567690,9,7,0.010078,0.013280,-0.098425,High,2022-2023
5,MBB,101609360,1609360.0,1.609360,6,6,-0.006658,-0.008560,-0.068131,High,2022-2023
0,NVL,103285500,3285500.0,3.285500,7,6,0.001735,0.002762,-0.063847,Low,2022-2023
1,REE,96529470,-3470530.0,-3.470530,9,9,-0.032472,-0.043669,-0.115143,Low,2022-2023
2,SBT,104998160,4998160.0,4.998160,8,7,0.012075,0.017478,-0.118572,Low,2022-2023
3,SSI,97988360,-2011640.0,-2.011640,7,7,-0.017229,-0.024991,-0.134977,Low,2022-2023


# gop 2 cai

In [86]:
merged_all = pd.concat([merged, merged_2022])
merged_all= merged_all.reset_index()
merged_all['total_trades']= merged_all['Number of Buying Trades'] + merged_all['Number of Selling Trades']
merged_all.to_csv('macd kelly.csv', index=False)

In [87]:
sell_fraction =1

In [88]:
a = simulate_investment('NVL', win_rate, loss_rate, mean_profit, mean_loss, \
                                     sell_fraction, start_date='2019-01-02', end_date='2023-01-02', f_star=f)


Time range is 1461 days. Looping through 5 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}


In [90]:
kelly = pd.read_csv('macd kelly.csv')
no_kelly = pd.read_csv('macd no kelly.csv')
kelly['Kelly'] = 'Yes'
no_kelly['Kelly'] = 'No'
double_merged = pd.concat([kelly, no_kelly])
double_merged.to_excel('macd.xlsx', index=False)